In [1]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()


In [ ]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown',
                                             options=[{'label': 'All Sites', 'value': 'ALL'},
                                                      {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                      {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                      {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                      {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                     ],
                                             value='ALL',
                                             placeholder="Select Launch Site here",
                                             searchable=True
                                            ),
                                ##for the drop down,i kept value and label the same so callback function is simple
                                html.Br(),
                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                
                                # Function decorator to specify function input and output

                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(id='payload-slider',min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       1000: '1000',
                                                       2000: '2000',
                                                       3000: '3000',
                                                       4000: '4000',
                                                      5000: '5000',
                                                       6000: '6000',
                                                      7000: '7000',
                                                       8000: '8000',
                                                       9000: '9000',
                                                      10000:'10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
# Function decorator to specify function input and output
@app.callback( Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
        names='Launch Site', 
        title='Total Successful Launches Of All Launch Sites')
        return fig
    else:
        fig=px.pie(spacex_df[spacex_df['Launch Site']==entered_site],
                   names='class')
        return fig
            
        
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(site_dropdown, payload_slider):
    spacex_df1=spacex_df[spacex_df["Payload Mass (kg)"].between(payload_slider[0],payload_slider[1])]
    if site_dropdown=='ALL':
        fig=px.scatter(spacex_df1,x="Payload Mass (kg)",y="class",color="Booster Version Category")
        return fig
    else:
        fig=px.scatter(spacex_df1[spacex_df1["Launch Site"]== site_dropdown],
                       x="Payload Mass (kg)",y="class",color="Booster Version Category")
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False)


In [ ]:
spacex_df1=spacex_df[spacex_df["Payload Mass (kg)"].between(2500,7500)]
spacex_df1

**Finding Insights Visually**

Now with the dashboard completed, you should be able to use it to analyze SpaceX launch data, and answer the following questions:

- Which site has the largest successful launches?
- Which site has the highest launch success rate?
- Which payload range(s) has the highest launch success rate?
- Which payload range(s) has the lowest launch success rate?
- Which F9 Booster version (v1.0, v1.1, FT, B4, B5, etc.) has the highest launch success rate?

- Launch Site 'KSC LC-39A' has the largest successful launches making 41.7%(10 Launches) of the total successful launches.
- Launch Site 'KSC LC-39A' has the highest success rate at 76.9%.
- Payload ranges 4600-5200kg and 3000-4000kg have the highest launch success rate
- Payload ranges 6000-8000kg and 500-2000kg have the lowest launch success rate
- Booster version B5 has the highest launch success rate.